### Définition de la fonction de scrapping des stats twitter

In [1]:
from bs4 import BeautifulSoup
import requests
import webbrowser
import time
import pyautogui
from datetime import datetime
import pandas as pd

In [2]:
def get_data(req):
    soup = BeautifulSoup(req.text, "lxml")
    stats=soup.find_all("table",{"class":"condensed-table"})[1].find_all("td")
    stats2=soup.find_all("table",{"class":"condensed-table"})[2].find_all("td")
    word=(soup.find_all("p",{"id":"topics-cloud"})[0]).find_all("a",{"style":"font-size: 22pt;"})[0].text
    tags=soup.find_all("a",{"target":"blank"})
    L=stats+stats2
    L=[s.text.replace('\t','').replace('\n','') for s in L]
    col=L[0::2][0:-2]
    val=L[1::2][0:-2]
    val=[(s.replace("/","")).replace(",","")+" " for s in val]
    val=[[float(s[0:s.find(" ")])  for s in val]+[word]]
    col.append("Mot prefere")
    df = pd.DataFrame(val, columns = col)
    return(df)
    
    

In [3]:
def stats_tweets(username):
    for i in range(len(username)):
        erreur=0
        now = datetime.now()
        now=now.strftime("%d/%m/%Y")
        name=username[i]
        webbrowser.open('https://foller.me/'+name+'#overview')
        #req = requests.get('https://foller.me/'+name+'#overview')
        time.sleep(5)
        req = requests.get('https://foller.me/'+name+'#overview')
        try:
            #On suppose qu'au bout de 5s la barre de chargement est terminée
            line=get_data(req)
            line['twitter']=name
            line['date_scrap']=now
        except:
            time.sleep(5)
            #Si ce n'est pas le cas, on attends à nouveau 5s et on attends
            req = requests.get('https://foller.me/'+name+'#overview')
            try:
                line=get_data(req)
                line['twitter']=name
                line['date_scrap']=now
            except:
                time.sleep(5)
                #On fait une dernière tentative 5s après
                req = requests.get('https://foller.me/'+name+'#overview')
                try:
                    line=get_data(req)
                    line['twitter']=name
                    line['date_scrap']=now
                except:
                    #Au bout de environ 15s, on considère qu'il y a une erreur sur le compte twitter donc on abandone 
                    print("Erreur : "+name)
                    erreur=1
        pyautogui.hotkey('ctrl', 'w')
        #Si on a erreur==1 alors on ne va pas ajouter la ligne dans aucun des cas
        if i ==0 and erreur==0:
            df=line #On crée le tableau pour le premier twitter
        elif erreur==0:
            df=pd.concat([df, line]) #Pour les twitter suivant, on concat en dessous
    df=df.reset_index().drop(['index'],axis=1).rename(columns={"Replies":"Replies/100derniers","Tweets with @mentions":"Tweets with @mentions/100derniers","Tweets with #hashtags":"Tweets with #hashtags/100derniers","Retweets":"Retweets/100","Tweets with links":"Tweets with links/100derniers","Tweets with media":"Tweets with media/100derniers"})
    return(df.reset_index().drop(['index'],axis=1))

### Utilisation sur les Twitters des députés

In [4]:
L_twitter=pd.read_csv("caract_deputes.csv",";")["twitter"].dropna().tolist()

C:\Users\Paul\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only
  exec(code_obj, self.user_global_ns, self.user_ns)


In [5]:
#Pour tester sur 5 députés (assez rapide)
tab=stats_tweets(L_twitter[:5])
tab

,Tweets,Followers,Following,Followers ratio,Listed,Replies/100derniers,Tweets with @mentions/100derniers,Tweets with #hashtags/100derniers,Retweets/100,Tweets with links/100derniers,Tweets with media/100derniers,Mot prefere,twitter,date_scrap
0,3681.0,5662.0,1023.0,5.53,131.0,1.0,55.0,49.0,8.0,92.0,0.0,modle,CedricRoussel06,01/01/2022
1,4956.0,4653.0,1215.0,3.83,82.0,5.0,60.0,13.0,15.0,82.0,1.0,cest,f_personne,01/01/2022
2,6304.0,10539.0,1264.0,8.34,178.0,1.0,95.0,21.0,92.0,8.0,14.0,cest,guillaumekasba,01/01/2022
3,6427.0,2594.0,1287.0,2.02,113.0,7.0,73.0,15.0,45.0,44.0,15.0,sant,Cisaacsibille,01/01/2022
4,3065.0,3773.0,812.0,4.65,130.0,6.0,77.0,51.0,46.0,53.0,3.0,cest,g_vuilletet,01/01/2022


#### Sur tous les députés

In [6]:
tab=stats_tweets(L_twitter)
tab=tab.merge(pd.read_csv("caract_deputes.csv",";")[['twitter','nom','DEPSIR']],on="twitter",how='right').fillna(0)
tab['twitter']=tab['twitter'].replace(0,"NON")

Erreur : BrunoDuverge
Erreur : fmeunier19
Erreur : B_Peyrol
Erreur : josette_manin
Erreur : StTRAVERT


C:\Users\Paul\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only
  exec(code_obj, self.user_global_ns, self.user_ns)


In [7]:
tab.head(10)

,Tweets,Followers,Following,Followers ratio,Listed,Replies/100derniers,Tweets with @mentions/100derniers,Tweets with #hashtags/100derniers,Retweets/100,Tweets with links/100derniers,Tweets with media/100derniers,Mot prefere,twitter,date_scrap,nom,DEPSIR
0,3681.0,5662.0,1023.0,5.53,131.0,1.0,55.0,49.0,8.0,92.0,0.0,modle,CedricRoussel06,01/01/2022,Cédric Roussel,0603
1,4956.0,4653.0,1215.0,3.83,82.0,5.0,60.0,13.0,15.0,82.0,1.0,cest,f_personne,01/01/2022,Pascale Fontenel-Personne,7203
2,6304.0,10539.0,1264.0,8.34,178.0,1.0,95.0,21.0,92.0,8.0,14.0,cest,guillaumekasba,01/01/2022,Guillaume Kasbarian,2801
3,6427.0,2594.0,1287.0,2.02,113.0,7.0,73.0,15.0,45.0,44.0,15.0,sant,Cisaacsibille,01/01/2022,Cyrille Isaac-Sibille,6912
4,3065.0,3773.0,812.0,4.65,130.0,6.0,77.0,51.0,46.0,53.0,3.0,cest,g_vuilletet,01/01/2022,Guillaume Vuilletet,9502
5,15579.0,66884.0,2546.0,26.27,739.0,1.0,66.0,31.0,49.0,48.0,8.0,cette,faureolivier,01/01/2022,Olivier Faure,7711
6,2528.0,6050.0,2482.0,2.44,124.0,7.0,89.0,34.0,65.0,33.0,0.0,merci,p_a_raphan,01/01/2022,Pierre-Alain Raphan,9110
7,446.0,1616.0,426.0,3.79,0.0,0.0,76.0,35.0,73.0,26.0,1.0,dputs,eanthoine26,01/01/2022,Emmanuelle Anthoine,2604
8,11294.0,9011.0,1112.0,8.10,175.0,55.0,84.0,11.0,22.0,51.0,2.0,cest,AChristine_Lang,01/01/2022,Anne-Christine Lang,7510
9,1470.0,3934.0,488.0,8.06,161.0,2.0,56.0,27.0,32.0,67.0,0.0,loi,Lvichnievsky,01/01/2022,Laurence Vichnievsky,6303


In [8]:
tab.to_csv("stats_twitter.csv",";")